# Disc break: diffusion of heat in a disc due to friction

In [ ]:
import matplotlib.pyplot as plt
import pyvista
import ufl
import numpy as np

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc

# Define temporal parameters
t = 0  # Start time
T = 10.0  # Final time
num_steps = 500
dt = T / num_steps  # time step size
boundary_conditions = "Neumann"
diffusion_coefficient = 2

Import the mesh. The mesh has been previously created using gmsh.

In [ ]:
import meshio
import h5py

msh = meshio.read("tire.msh")
for cell in msh.cells:
    triangle_cells = cell.data
for key in msh.cell_data_dict["gmsh:physical"].keys():
    if key == "triangle":
        triangle_data = msh.cell_data_dict["gmsh:physical"][key]
domain =meshio.Mesh(points=msh.points,
                           cells=[("triangle", triangle_cells)],
                           cell_data={"name_to_read":[triangle_data]})
meshio.write("diffusion.xdmf",domain)

In [ ]:
import dolfinx
from dolfinx.io import XDMFFile

with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "diffusion.xdmf", "r") as xdmf:
    domain = xdmf.read_mesh(name="Grid")

Visualize the mesh

In [ ]:
from dolfinx.plot import vtk_mesh
from dolfinx import fem, io, plot

pyvista.start_xvfb()
plotter = pyvista.Plotter()
grid = pyvista.UnstructuredGrid(*vtk_mesh(domain, domain.topology.dim))
num_local_cells = domain.topology.index_map(domain.topology.dim).size_local
#grid.cell_data["Marker"] = ct.values[ct.indices < num_local_cells]
#grid.set_active_scalars("Marker")
actor = plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    cell_tag_fig = plotter.screenshot("domain.png")

Define the function space

In [ ]:
V = fem.FunctionSpace(domain, ("Lagrange", 1))

In [ ]:
import numpy as np
# Create initial condition
#def initial_condition(x):
#    return 1/(1+(x[0] + x[1])**2)
#initial_condition = fem.Constant(domain, PETSc.ScalarType(0))

u_n = fem.Function(V)
u_n.name = "u_n"
#u_n.interpolate(initial_condition)
u_n.interpolate(lambda x: np.full(x.shape[1], 0, dtype=np.float64))

# Create boundary condition
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
if boundary_conditions == 'Dirichlet':
    bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

## Time-dependent output
To visualize the solution in an external program such as Paraview, we create a an `XDMFFile` which we can store multiple solutions in. The main advantage with an XDMFFile, is that we only need to store the mesh once, and can append multiple solutions to the same grid, reducing the storage space.
The first argument to the XDMFFile is which communicator should be used to store the data. As we would like one output, independent of the number of processors, we use the `COMM_WORLD`. The second argument is the file name of the output file, while the third argument is the state of the file,
this could be read (`"r"`), write (`"w"`) or append (`"a"`).

In [ ]:
xdmf = io.XDMFFile(domain.comm, "diffusion.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(lambda x: np.full(x.shape[1], 0, dtype=np.float64))
xdmf.write_function(uh, t)

## Variational problem and solver
In this case we make use of Constant classes to define the source term f, so that we will later be able to change them during the evolution of the process.

In [ ]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)

import ufl
from dolfinx import default_scalar_type

D = fem.Constant(domain, default_scalar_type(diffusion_coefficient))

x = ufl.SpatialCoordinate(domain)
R = fem.Constant(domain, default_scalar_type(8))
theta = fem.Constant(domain, default_scalar_type(0))
alpha = fem.Constant(domain, default_scalar_type(10))
sigma = fem.Constant(domain, default_scalar_type(1))
#f = 1 / ufl.exp(1+(x[0]-1)**2 + (x[1])**2)
f = alpha * ufl.exp(-((x[0] - R*ufl.cos(theta))**2 / (2 * sigma**2) + (x[1] - R*ufl.sin(theta))**2 / (2 * sigma**2))) - 0.01

a = u * v * ufl.dx + D * dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
if boundary_conditions == "Dirichlet":
    L = (u_n + dt * f) * v * ufl.dx 
elif boundary_conditions == "Neumann":
    g = -0.01
    L = (u_n + dt * f) * v * ufl.dx + g * v * ufl.ds

## Preparing linear algebra structures for time dependent problems

In [ ]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [ ]:
Q = fem.FunctionSpace(domain, ("Lagrange", 5))
expr = fem.Expression(f, Q.element.interpolation_points())
load = fem.Function(Q)
load.interpolate(expr)

We observe that the left hand side of the system, the matrix $A$ does not change from one time step to another, thus we only need to assemble it once. However, the right hand side, which is dependent on the previous time step `u_n`, we have to assemble it every time step. Therefore, we only create a vector `b` based on `L`, which we will reuse at every time step.

In [ ]:
#A = assemble_matrix(bilinear_form, bcs=[bc])
A = assemble_matrix(bilinear_form)
A.assemble()
b = create_vector(linear_form)

## Using petsc4py to create a linear solver
As we have already assembled `a` into the matrix `A`, we can no longer use the `dolfinx.fem.petsc.LinearProblem` class to solve the problem. Therefore, we create a linear algebra solver using PETSc, and assign the matrix `A` to the solver, and choose the solution strategy.

In [ ]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

## Visualization of time dependent problem using pyvista
We use the DOLFINx plotting functionality, which is based on pyvista to plot the solution at every $15$th time step. We would also like to visualize a colorbar reflecting the minimal and maximum value of $u$ at each time step. We use the following convenience function `plot_function` for this:

In [ ]:
import matplotlib as mpl
pyvista.start_xvfb()

grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))

plotter_3D = pyvista.Plotter()
plotter_3D.open_gif("u_time_3D.gif", fps=10)
plotter_2D = pyvista.Plotter()
plotter_2D.open_gif("u_time_2D.gif", fps=10)

grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)

colormap = mpl.colormaps.get_cmap("coolwarm").resampled(50)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)

renderer = plotter_3D.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=colormap, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])
renderer = plotter_2D.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=colormap, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])
plotter_2D.view_xy()

## Updating the solution and right hand side per time step
To be able to solve the variation problem at each time step, we have to assemble the right hand side and apply the boundary condition before calling
`solver.solve(b, uh.vector)`. We start by resetting the values in `b` as we are reusing the vector at every time step.
The next step is to assemble the vector, calling `dolfinx.fem.petsc.assemble_vector(b, L)` which means that we are assemble the linear for `L(v)` into the vector `b`. Note that we do not supply the boundary conditions for assembly, as opposed to the left hand side.
This is because we want to use lifting to apply the boundary condition, which preserves symmetry of the matrix $A$ if the bilinear form $a(u,v)=a(v,u)$ without Dirichlet boundary conditions.
When we have applied the boundary condition, we can solve the linear system and update values that are potentially shared between processors.
Finally, before moving to the next time step, we update the solution at the previous time step to the solution at this time step.

In [ ]:
t = 0
for i in range(num_steps):
    t += dt
    theta.value = 2 * np.pi * t / (0.1*T)

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    if boundary_conditions == 'Dirichlet':
        apply_lifting(b, [bilinear_form], [[bc]])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, uh.vector)
    uh.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh.x.array

    # Write solution to file
    xdmf.write_function(uh, t)
    # Update plot
    warped = grid.warp_by_scalar("uh", factor=1)
    plotter_3D.update_coordinates(warped.points.copy(), render=False)
    plotter_3D.update_scalars(uh.x.array, render=False)
    plotter_3D.write_frame()
    plotter_2D.update_scalars(uh.x.array, render=False)
    plotter_2D.write_frame()
plotter_3D.close()
plotter_2D.close()
xdmf.close()

In [ ]:
from IPython.display import display, Image

# Path to your GIF file
gif_path = './u_time_2D.gif'

# Display the GIF
with open(gif_path, 'rb') as f:
    display(Image(data=f.read(), format='png'))

# Path to your GIF file
gif_path = './u_time_3D.gif'

# Display the GIF
with open(gif_path, 'rb') as f:
    display(Image(data=f.read(), format='png'))

## Animation with Paraview
We can also use Paraview to create an animation. We open the file in paraview with `File->Open`, and then press `Apply` in the properties panel.

Then, we add a time-annotation to the figure, pressing: `Sources->Alphabetical->Annotate Time` and `Apply` in the properties panel. It Is also a good idea to select an output resolution, by pressing `View->Preview->1280 x 720 (HD)`.

Then finally, click `File->Save Animation`, and save the animation to the desired format, such as `avi`, `ogv` or a sequence of `png`s. Make sure to set the framerate to something, sensible, in the range of $5-10$ frames per second.